# Requetes Séance 1

In [3]:
import sqlite3
import pandas as pd

In [4]:
conn = sqlite3.connect("C:/Users/rhofr/OneDrive/Bureau/but3/sae nosql/seance 3/ClassicModel.sqlite")

## 1. Lister les clients n’ayant jamais effecuté une commande ;

In [5]:
customers_1 = pd.read_sql_query(
    """
    SELECT Customers.customerNumber, Customers.customerName     
    FROM Customers
    LEFT JOIN Payments ON Customers.customerNumber = Payments.customerNumber
    WHERE Payments.customerNumber IS NULL
    """, conn
)

print(customers_1)

    customerNumber                    customerName
0              125              Havel & Zbyszek Co
1              168          American Souvenirs Inc
2              169               Porto Imports Co.
3              206      Asian Shopping Network, Co
4              223                 Natürlich Autos
5              237                   ANG Resellers
6              247        Messner Shopping Network
7              273               Franken Gifts, Co
8              293               BG&E Collectables
9              303                Schuyler Imports
10             307                Der Hund Imports
11             335       Cramer Spezialitäten, Ltd
12             348           Asian Treasures, Inc.
13             356            SAR Distributors, Co
14             361                 Kommission Auto
15             369          Lisboa Souveniers, Inc
16             376           Precious Collectables
17             409  Stuttgart Collectable Exchange
18             443        Feuer

#### Objectif de la démarche : Identifier les clients inactifs pour lancer des campagnes de relance ciblées. 
#### Explication de la démarche :
#### On effectue un left join entre les tables "Customers" et "Payments" avec le numéro du client (customerNumber) comme clé de jointure dans le but d'avoir tous les clients, même ceux qui ne sont pas présents dans la table "Payments". On filtre ensuite les résultats de cette jointure pour ne garder que les clients dont le numéro client n’apparaît pas dans la table Payments, c’est-à-dire les clients qui n’ont jamais effectué de paiement (donc de commande). On décide de n'afficher que le numéro et le nom du client

## 2. Pour chaque employé, le nombre de clients, le nombre de commandes et le montant total de celles-ci ;

In [6]:
employes = pd.read_sql_query(
    """
    SELECT 
        e.employeeNumber, e.firstName, e.lastName,
        COUNT(DISTINCT c.customerNumber) AS nb_clients,
        COUNT(DISTINCT o.orderNumber) AS nb_commandes,
        SUM(od.quantityOrdered * od.priceEach) AS montant_total
    FROM 
        Employees e
    LEFT JOIN 
        Customers c ON e.employeeNumber = c.salesRepEmployeeNumber
    LEFT JOIN 
        Orders o ON c.customerNumber = o.customerNumber
    LEFT JOIN 
        OrderDetails od ON o.orderNumber = od.orderNumber
    GROUP BY 
        e.employeeNumber, e.firstName, e.lastName;
    """, conn
)

print(employes)

    employeeNumber firstName   lastName  nb_clients  nb_commandes  \
0             1002     Diane     Murphy           0             0   
1             1056      Mary  Patterson           0             0   
2             1076      Jeff   Firrelli           0             0   
3             1088   William  Patterson           0             0   
4             1102    Gerard     Bondur           0             0   
5             1143   Anthony        Bow           0             0   
6             1165    Leslie   Jennings           6            34   
7             1166    Leslie   Thompson           6            14   
8             1188     Julie   Firrelli           6            14   
9             1216     Steve  Patterson           6            18   
10            1286  Foon Yue      Tseng           7            17   
11            1323    George     Vanauf           8            22   
12            1337      Loui     Bondur           6            20   
13            1370    Gerard  Hern

#### Objectif de la démarche : Évaluer la performance des employés.
#### Explication de la démarche :
#### Jointure des tables :

#### On effectue une série de left join entre les tables Employees, Customers, Orders, et OrderDetails dans le but d'inclure tous les employés de la base de données : 
#### - un left join entre les tables Employees et Customers en utilisant le employeeNumber comme clé de jointure dans le but d'avoir tous les employés, même ceux sans clients associés,
#### - un left join entre les tables Customers et Orders en utilisant customerNumber comme clé de jointure dans le but d'avoir tous les clients, même ceux sans commandes associées,
#### - un left join entre les tables Orders et OrderDetails en utilisant orderNumber comme clé de jointure dans le but d'avoir toutes les commandes, même celles sans détails associés.
#### Groupement des résultats :

#### On groupe les résultats par le numéro (employeeNumber), le nom (firstName), et le prénom (lastName) de l'employé pour obtenir des statistiques agrégées pour chaque employé individuellement.
#### Calculs :

#### On décide d'afficher le numéro de l'employé (employeeNumber) et le nom complet de l’employé (firstName || ' ' || lastName) pour identifier chaque employé de manière unique et lisible.
#### On utilise la fonction COUNT(DISTINCT c.customerNumber) pour compter le nombre de clients distincts associés à chaque employé : on nomme la colonne "nb_clients"
#### On utilise la fonction COUNT(DISTINCT o.orderNumber) pour compter le nombre de commandes distinctes associées à chaque employé : on nomme la colonne "nb_commandes"
#### On utilise la fonction SUM(od.quantityOrdered * od.priceEach) pour calculer le montant total des commandes associées à chaque employé qui fait la somme de chaque produit quantité*prix  pour chaque commandes associées à chaque employé. On nomme la colonne "montant_total".


## 3. Idem pour chaque bureau (nombre de clients, nombre de commandes et montant total), avec en plus le nombre de clients d’un pays différent, s’il y en a ;

In [7]:
office_stats = pd.read_sql_query(
    """
    SELECT 
        o.officeCode,
        o.city,
        COUNT(DISTINCT c.customerNumber) AS nb_clients,
        COUNT(DISTINCT ord.orderNumber) AS nb_commandes,
        SUM(od.quantityOrdered * od.priceEach) AS montant_total,
        COUNT(DISTINCT CASE WHEN c.country != o.country THEN c.customerNumber END) AS nb_clients_etrangers
    FROM 
        Offices o
    LEFT JOIN 
        Employees e ON o.officeCode = e.officeCode
    LEFT JOIN 
        Customers c ON e.employeeNumber = c.salesRepEmployeeNumber
    LEFT JOIN 
        Orders ord ON c.customerNumber = ord.customerNumber
    LEFT JOIN 
        OrderDetails od ON ord.orderNumber = od.orderNumber
    GROUP BY 
        o.officeCode, o.city
    ORDER BY 
        o.officeCode;
    """, conn
)

print(office_stats)

  officeCode           city  nb_clients  nb_commandes  montant_total  \
0        1.0  San Francisco          12            48     1588293.29   
1        2.0         Boston          12            32      987774.17   
2        3.0            NYC          15            39     1275993.95   
3        4.0          Paris          29           106     3404055.56   
4        5.0          Tokyo           5            16      503957.58   
5        6.0         Sydney          10            38     1281705.83   
6        7.0         London          17            47     1604168.80   

   nb_clients_etrangers  
0                     0  
1                     0  
2                     3  
3                    17  
4                     3  
5                     5  
6                    12  


#### Objectif de la démarche : Analyser les performances des bureaux et leur portée internationale.
#### Explication de la démarche :
#### Jointure des tables : 
#### On effectue une série de left join pour avoir tous les bureaux, même ceux sans employés, clients, ou commandes associés.
#### - entre Offices et Employees sur officeCode pour inclure tous les bureaux,
#### - entre Employees et Customers sur employeeNumber pour inclure tous les employés,
#### - entre Customers et Orders sur customerNumber pour inclure tous les clients,
#### - entre Orders et OrderDetails sur orderNumber pour inclure toutes les commandes.

#### Groupement des résultats :
#### On groupe les résultats par le numéro du bureau (officeCode) et la ville (city) pour obtenir les statistiques agrégées pour chaque bureau.

#### Séléction des colonnes et calculs : 
#### On sélectionne les colonnes officeCode et city pour identifier chaque bureau par son numéro et sa ville.
#### On utilise COUNT(DISTINCT c.customerNumber) pour compter le nombre de clients distincts associés à chaque bureau,
#### On utilise COUNT(DISTINCT ord.orderNumber) pour compter le nombre de commandes distinctes associées à chaque bureau,
#### On utilise SUM(od.quantityOrdered * od.priceEach) pour calculer le montant total des commandes associées à chaque bureau de la même manière que dans la question 2), 
#### On utilise COUNT(DISTINCT CASE WHEN c.country != o.country THEN c.customerNumber END) pour compter le nombre de clients distincts d’un pays différent en mettant en place une condition : si le pays du client (c.countru) est différent du pays du bureau (o.country), le numéro du client est retourné. 

## 4. Pour chaque produit, donner le nombre de commandes, la quantité totale commandée, et le nombre de clients différents ;

In [8]:
stats_produits = pd.read_sql_query(
    """
    SELECT Products.productCode, Products.productName, COUNT(DISTINCT OrderDetails.orderNumber) AS numberOfOrders,
           SUM(OrderDetails.quantityOrdered) AS totalQuantity,
           COUNT(DISTINCT Orders.customerNumber) AS numberOfClients
    FROM Products
    LEFT JOIN OrderDetails ON Products.productCode = OrderDetails.productCode
    LEFT JOIN Orders ON OrderDetails.orderNumber = Orders.orderNumber
    GROUP BY Products.productCode
    """,
    conn
)
print("Stats par produit:\n", stats_produits)

Stats par produit:
     productCode                            productName  numberOfOrders  \
0      S10_1678  1969 Harley Davidson Ultimate Chopper              28   
1      S10_1949               1952 Alpine Renault 1300              28   
2      S10_2016                  1996 Moto Guzzi 1100i              28   
3      S10_4698   2003 Harley-Davidson Eagle Drag Bike              28   
4      S10_4757                    1972 Alfa Romeo GTA              28   
..          ...                                    ...             ...   
105   S700_3505                            The Titanic              27   
106   S700_3962                         The Queen Mary              27   
107   S700_4002              American Airlines: MD-11S              28   
108    S72_1253                       Boeing X-32A JSF              28   
109    S72_3212                             Pont Yacht              27   

     totalQuantity  numberOfClients  
0           1026.0               26  
1            96

#### Objectif de la démarche : Identifier les produits les plus populaires et les tendances d’achat.
#### Explication de la démarche :
#### - Jointure des tables :
#### On relie les produits (`Products`) avec les commandes (`Orders`) et les détails de commande (`OrderDetails`). 
#### Colonnes calculées : 
#### `COUNT(DISTINCT OrderDetails.orderNumber)` pour le nombre de commandes ayant inclus le produit. 
#### `SUM(OrderDetails.quantityOrdered)` pour la quantité totale commandée. 
#### `COUNT(DISTINCT Orders.customerNumber)` pour le nombre de clients ayant commandé ce produit. 
####Groupement : Les résultats sont regroupés par produit (`productCode`). 

## 5. Donner le nombre de commande pour chaque pays, ainsi que le montant total des commandes et le montant total payé : on veut conserver les clients n’ayant jamais commandé dans le résultat final ;

In [9]:
stats_pays = pd.read_sql_query(
    """
    SELECT Customers.country, COUNT(DISTINCT Orders.orderNumber) AS numberOfOrders,
           SUM(OrderDetails.quantityOrdered * OrderDetails.priceEach) AS totalAmount,
           SUM(Payments.amount) AS totalPaid
    FROM Customers
    LEFT JOIN Orders ON Customers.customerNumber = Orders.customerNumber
    LEFT JOIN OrderDetails ON Orders.orderNumber = OrderDetails.orderNumber
    LEFT JOIN Payments ON Customers.customerNumber = Payments.customerNumber
    GROUP BY Customers.country
    """,
    conn
)
print("Stats par pays:\n", stats_pays)

Stats par pays:
          country  numberOfOrders  totalAmount     totalPaid
0      Australia              19   2182269.38  2.482541e+07
1        Austria               7    606187.59  4.090982e+06
2        Belgium               7    283705.44  1.931535e+06
3         Canada               7    448157.12  4.487022e+06
4        Denmark               7    781357.50  7.001114e+06
5        Finland               9    988745.73  1.009620e+07
6         France              37   3160296.75  3.141444e+07
7        Germany               7    576293.44  4.971661e+06
8      Hong Kong               2     48784.36  7.805498e+05
9        Ireland               2    115512.86  9.241029e+05
10        Israel               0          NaN           NaN
11         Italy              10    945208.16  1.324310e+07
12         Japan               6    496898.36  4.837611e+06
13   Netherlands               0          NaN           NaN
14   New Zealand              15   1736137.04  1.710337e+07
15        Norway       

#### Objectif de la démarche : Avoir une vue d’ensemble par pays. 
#### Explication de la démarche :
#### - Jointure des tables : On relie les clients (`Customers`) avec les commandes (`Orders`), les détails de commande (`OrderDetails`), et les paiements (`Payments`). 
#### Colonnes calculées : 
#### `COUNT(DISTINCT Orders.orderNumber)` pour le nombre de commandes par pays. 
#### `SUM(OrderDetails.quantityOrdered * OrderDetails.priceEach)` pour le chiffre d’affaires total. 
#### `SUM(Payments.amount)` pour le total des paiements effectués. 
#### Groupement : Les résultats sont regroupés par pays (`country`). 

## 6. On veut la table de contigence du nombre de commande entre la ligne de produits et le pays du client ;

In [10]:
contingence_commandes = pd.read_sql_query(
    """
    SELECT Products.productLine, Customers.country, COUNT(DISTINCT OrderDetails.orderNumber) AS numberOfOrders
    FROM Products
    LEFT JOIN OrderDetails ON Products.productCode = OrderDetails.productCode
    LEFT JOIN Orders ON OrderDetails.orderNumber = Orders.orderNumber
    LEFT JOIN Customers ON Orders.customerNumber = Customers.customerNumber
    GROUP BY Products.productLine, Customers.country
    """,
    conn
)
contingence_commandes_pivot = contingence_commandes.pivot(index='productLine', columns='country', values='numberOfOrders')
print("Table de contingence (nombre de commandes):\n", contingence_commandes_pivot)

Table de contingence (nombre de commandes):
 country           NaN  Australia  Austria  Belgium  Canada  Denmark  Finland  \
productLine                                                                    
Classic Cars      0.0       12.0      5.0      2.0     6.0      5.0      9.0   
Motorcycles       NaN        6.0      1.0      NaN     1.0      NaN      2.0   
Planes            NaN        5.0      2.0      1.0     2.0      1.0      2.0   
Ships             NaN        2.0      1.0      3.0     2.0      3.0      2.0   
Trains            NaN        1.0      NaN      3.0     NaN      2.0      1.0   
Trucks and Buses  NaN        5.0      1.0      NaN     3.0      1.0      2.0   
Vintage Cars      NaN       14.0      4.0      5.0     5.0      3.0      2.0   

country           France  Germany  Hong Kong  ...  Japan  New Zealand  Norway  \
productLine                                   ...                               
Classic Cars        23.0      4.0        NaN  ...    3.0          9.0   

#### Objectif de la démarche : Identifier les tendances géographiques par ligne de produits. 
#### Explication de la démarche :
#### - Jointure des tables : On relie les produits (`Products`) avec les détails de commande (`OrderDetails`). 
#### - Colonnes calculées : `AVG(OrderDetails.priceEach - Products.buyPrice)` calcule la marge moyenne de chaque produit. 
#### - Tri : Les produits sont triés par marge moyenne décroissante avec `ORDER BY avgMargin DESC`. 
#### - Limitation : On conserve uniquement les 10 premiers produits grâce à `LIMIT 10`. 

## 7. On veut la même table croisant la ligne de produits et le pays du client, mais avec le montant total payé dans chaque cellule ;

In [11]:
contingence_paiements = pd.read_sql_query(
    """
    SELECT Products.productLine, Customers.country, SUM(Payments.amount) AS totalPaid
    FROM Products
    LEFT JOIN OrderDetails ON Products.productCode = OrderDetails.productCode
    LEFT JOIN Orders ON OrderDetails.orderNumber = Orders.orderNumber
    LEFT JOIN Customers ON Orders.customerNumber = Customers.customerNumber
    LEFT JOIN Payments ON Customers.customerNumber = Payments.customerNumber
    GROUP BY Products.productLine, Customers.country
    """,
    conn
)
contingence_paiements_pivot = contingence_paiements.pivot(index='productLine', columns='country', values='totalPaid')
print("Table de contingence (montant total payé):\n", contingence_paiements_pivot)

Table de contingence (montant total payé):
 country           NaN   Australia     Austria    Belgium      Canada  \
productLine                                                            
Classic Cars      NaN  7504795.97  1884419.42  166880.87   774924.01   
Motorcycles       NaN  4112158.58   413377.90        NaN    74634.85   
Planes            NaN  3619902.87   496053.48   33440.10   746348.50   
Ships             NaN   157812.92   330702.32  699045.80   840149.72   
Trains            NaN    59469.12         NaN  166561.24         NaN   
Trucks and Buses  NaN  2824360.08   261319.50        NaN  1030187.22   
Vintage Cars      NaN  6546906.20   705109.08  865607.04  1020778.08   

country              Denmark     Finland      France     Germany  Hong Kong  \
productLine                                                                   
Classic Cars      3678313.22  4219809.84  8802981.74  3414282.01        NaN   
Motorcycles              NaN  1457098.02  6226561.02   300919.74   487

#### Objectif de la démarche : Identifier les produits les plus rentables.
#### Explication de la démarche :
#### Jointure des tables : On relie les produits (`Products`), les commandes (`Orders`), et les clients (`Customers`). 
#### - Filtrage : On utilise la condition `WHERE OrderDetails.priceEach < Products.buyPrice` pour détecter les ventes à perte. 
#### - Colonnes sélectionnées : 
#### - `productCode` et `productName` pour identifier le produit. 
#### - `customerNumber` et `customerName` pour identifier le client associé à la vente. 
#### - `priceEach` et `buyPrice` pour comparer le prix de vente et le coût. 

## 8. Donner les 10 produits pour lesquels la marge moyenne est la plus importante (cf buyPrice et priceEach) ;

In [12]:
marges_produits = pd.read_sql_query(
    """
    SELECT Products.productCode, Products.productName, AVG(OrderDetails.priceEach - Products.buyPrice) AS avgMargin
    FROM Products
    LEFT JOIN OrderDetails ON Products.productCode = OrderDetails.productCode
    GROUP BY Products.productCode
    ORDER BY avgMargin DESC
    LIMIT 10
    """,
    conn
)
print("Top 10 produits avec marge moyenne la plus importante:\n", marges_produits)

Top 10 produits avec marge moyenne la plus importante:
   productCode                           productName  avgMargin
0    S10_1949              1952 Alpine Renault 1300  99.006429
1    S10_4698  2003 Harley-Davidson Eagle Drag Bike  95.235000
2    S18_3232           1992 Ferrari 360 Spider red  83.334906
3    S12_2823                      2002 Suzuki XREO  83.201429
4    S18_2795                1928 Mercedes-Benz SSK  82.696786
5    S12_1108                     2001 Ferrari Enzo  81.043704
6    S12_3891                      1969 Ford Falcon  77.335926
7    S18_3685        1948 Porsche Type 356 Roadster  72.636800
8    S18_2870          1999 Indy 500 Monte Carlo SS  71.794400
9    S18_1749              1917 Grand Touring Sedan  70.432800


## 9. Lister les produits (avec le nom et le code du client) qui ont été vendus à perte : Si un produit a été dans cette situation plusieurs fois, il doit apparaître plusieurs fois, Une vente à perte arrive quand le prix de vente est inférieur au prix d’achat ;

In [13]:
produits_perte = pd.read_sql_query(
    """
    SELECT Products.productCode, Products.productName, Orders.customerNumber, Customers.customerName,
           OrderDetails.priceEach, Products.buyPrice
    FROM Products
    LEFT JOIN OrderDetails ON Products.productCode = OrderDetails.productCode
    LEFT JOIN Orders ON OrderDetails.orderNumber = Orders.orderNumber
    LEFT JOIN Customers ON Orders.customerNumber = Customers.customerNumber
    WHERE OrderDetails.priceEach < Products.buyPrice
    """,
    conn
)
print("Produits vendus à perte:\n", produits_perte)

Produits vendus à perte:
    productCode                          productName  customerNumber  \
0     S10_4962               1962 LanciaA Delta 16V             363   
1     S18_2957                   1934 Ford V8 Coupe             363   
2     S18_3136  18th Century Vintage Horse Carriage             363   
3     S12_3148                   1969 Corvair Monza             181   
4     S18_2319               1964 Mercedec Tour Bus             181   
..         ...                                  ...             ...   
74    S10_4962               1962 LanciaA Delta 16V             276   
75    S12_1666                       1958 Setra Bus             276   
76    S18_2949          1913 Ford Model T Speedster             276   
77    S18_2238       1998 Chrysler Plymouth Prowler             323   
78    S12_1108                    2001 Ferrari Enzo             250   

                    customerName  priceEach  buyPrice  
0   Online Diecast Creations Co.      61.99    103.42  
1   Onlin

In [14]:
conn.close()